# 🚌 Escenario Alternativo: 50% usan transporte público

Este cuaderno evalúa el impacto de una política hipotética que cambia el modo de transporte de la mitad de los trabajadores al autobús, y calcula la nueva huella de carbono total.

In [ ]:
# Cargar librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from src.emissions.model import estimate_emissions
sns.set(style='whitegrid')

In [ ]:
# Cargar datos existentes
df = pd.read_csv('data/processed/datos_movilidad_coordenadas.csv')
df = df.dropna(subset=['Distancia_km'])

## Crear columna de escenario: 50% aleatorio usan 'bus'

In [ ]:
import numpy as np
df['Modo_Alternativo'] = df['Modo_Transporte']
mask = np.random.rand(len(df)) < 0.5
df.loc[mask, 'Modo_Alternativo'] = 'bus'

## Recalcular emisiones para el escenario

In [ ]:
df['Emision_alternativa'] = df.apply(lambda row: estimate_emissions(row['Modo_Alternativo'], row['Distancia_km']) * 2 * 220, axis=1)

## Comparativa de emisiones por trabajador

In [ ]:
df['Reduccion_kg'] = df['Emision_anual_kgCO2'] - df['Emision_alternativa']

plt.figure(figsize=(10, 5))
df_sorted = df.sort_values('Reduccion_kg', ascending=False)
sns.barplot(x='Nombre', y='Reduccion_kg', data=df_sorted)
plt.title('Reducción estimada de CO₂ anual por trabajador (escenario 50% bus)')
plt.ylabel('kg CO₂ evitados por año')
plt.xlabel('Trabajador')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Emisiones totales: realidad vs escenario

In [ ]:
real_total = df['Emision_anual_kgCO2'].sum()
alt_total = df['Emision_alternativa'].sum()
print(f"Total actual: {real_total:.2f} kg CO₂")
print(f"Total escenario alternativo: {alt_total:.2f} kg CO₂")
print(f"Reducción total: {real_total - alt_total:.2f} kg CO₂")